In [ ]:
from os import getenv
from sys import getsizeof
from random import uniform
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

from hdwallet import HDWallet, BIP141HDWallet
from hdwallet.symbols import FOXD
from hdwallet.cryptocurrencies import get_cryptocurrency

from dotenv import load_dotenv
from mnemonic import Mnemonic
import pickledb
import wx

from src.RPC import Call
from src.WalletDB import Wallet
from src.Database import credentials, mnemonic, settings, sighash
from src.node_conf.Helper import Helper

load_dotenv()

In [ ]:
print(True) if 0 else print(False)

In [ ]:
class Application(wx.App):
    def OnInit(self):
        app = Helper(None).Show(True)
        return True
app = Application()
app.MainLoop()

In [ ]:
hdkeypath = "m/0'/0'/0'"
foxdWallet = BIP141HDWallet(FOXD, get_cryptocurrency(FOXD), hdkeypath)
foxdWallet.from_xprivate_key(getenv('XPRIV')) # type: ignore
foxdWallet.p2pkh_address()

In [ ]:
language = settings['language']
password = settings['password']
sighash['anyone']['all']

In [ ]:
# if walletdb.lenaddr < amount:
#     amount = amount
# walletdb.generate_addresses(amount)

def switch_account_wallet(): pass

# from concurrent.futures import ThreadPoolExecutor, as_completed
# with ThreadPoolExecutor(16) as executor:
#     for i in range(10): # walletdb.lenaddr//50
#         futures = [executor.submit(rpc.getaddressbalance, walletdb.generate_addresses(i))]
#         for future in as_completed(futures):
#             print(future.result())

In [ ]:
rpc = Call(getenv('RPCUSER'), getenv('RPCPASS'), 8770)

def vouts(**kwrgs): return kwrgs
def vins(txids: list, vouts: list): return [{'txid': t, 'vout': v} for t, v in zip(txids, vouts)]
def createtx(vins: list[dict], vouts: dict): return rpc.createrawtransaction(vins, vouts)
def calcfee(txhash: str, feerate=.01): return feerate * getsizeof(txhash) / 1000

wallet = walletdb.get_account(0, 0)
address = wallet.p2pkh_address()
priv_key = wallet.wif()
wallet = walletdb.get_account(0, 1)
chaddr = wallet.p2pkh_address()

def send_to(p2pkh: str | list[str]): return


def send_coin(p2pkh, wif):
    utxo = rpc.getaddressutxos(p2pkh)[0]
    txid = utxo['txid']
    vout = utxo['outputIndex']
    script = utxo['script']
    balance = round(utxo['satoshis'] / 1e8, 8)

    def calcfee():
        txhash = createtx([{'txid': txid, 'vout': vout}], {})
        sigtx = rpc.signrawtransaction(txhash, [{'txid': txid, 'vout': vout, 'scriptPubKey': script}], [wif], "ALL|ANYONECANPAY")
        bytesize = getsizeof(sigtx['hex'])
        return round(.01 * bytesize / 1000, 8) # .01 * rpc.decoderawtransaction(sigtx['hex'])['size'] / 1000

    # txhash = rpc.createrawtransaction([{'txid': txid, 'vout': vout}], {'FEXoWJvWszsofzwEv4rvKAj54u94jxk9KW': (balance//2)-txfee, chaddr:(balance//2)-txfee})
    # sigtx = rpc.signrawtransaction(txhash, [{'txid': txid, 'vout': vout, 'scriptPubKey': script}], [wif], "ALL|ANYONECANPAY")
    # if bool(sigtx['complete']): return rpc.sendrawtransaction(str(sigtx['hex']))
# print(rpc.decoderawtransaction(send_coin(address, priv_key)))
# print(send_coin(address, priv_key)) # 4bef8d65e7c223a3f015789cfb3bc250995893c9c9fd198e80e6891d006811e8 # FEXoWJvWszsofzwEv4rvKAj54u94jxk9KW

In [ ]:
rpc = Call(getenv('RPCUSER'), getenv('RPCPASS'), 8770)
# rpc.getaddressbalance(address)
rpc.decoderawtransaction()
rpc.gettransaction('4bef8d65e7c223a3f015789cfb3bc250995893c9c9fd198e80e6891d006811e8')



In [ ]:
rpc = Call(getenv('RPCUSER'), getenv('RPCPASS'), 8770)
wallet, account, amount = f'wallet.{0}', f'account.{3}', 1000
phrase = p if isinstance(p := getenv('MNEM'), str) else mnemonic[wallet][-1]
walletdb = Wallet('FOXD', wallet, account, phrase)
offset = 4000 # walletdb.lenaddrs
db = walletdb.jsonDB
gen_addrs = walletdb.generate_addresses
def get_addresses(amount, offset):
    with ThreadPoolExecutor(16) as executor:
        def addresses(): return executor.submit(gen_addrs, amount).result()
        if walletdb.lenaddrs >= amount + offset: 
            ret = [executor.submit(db.lget, account, i + offset) for i in range(amount)] 
            for futr in as_completed(ret): yield futr.result()
        else: return addresses()
addresses = list(get_addresses(amount, offset))

print(addresses[0], addresses[-1])

In [ ]:
walletdb = Wallet('FOXD', wallet, account, phrase)
walletdb.lenaddrs
# addresses

In [ ]:
with ThreadPoolExecutor(16) as executor:
    bals = [executor.submit(rpc.getaddressbalance, {'addresses': [address]}) for address in (addresses[0], addresses[-1])]
    for futr in as_completed(bals): print(futr.result())

In [ ]:
bal = rpc.getbalance('')
recipients = {address: round(bal/uniform(100000, 200000), 8) for address in addresses}
print(f'{bal}\n{recipients}')

In [ ]:
rpc.walletpassphrase(getenv('UNLOCK'), 60)

In [ ]:
tdb = pickledb.load('tx.json', True)
if not tdb.exists(wallet): tdb.lcreate(wallet)

tx = rpc.sendmany('', recipients, 1, '', addresses)
if not tx: print('There was no transaction.')
else: tdb.append(wallet, [tx])
if tx: print(f'{tx}\n{recipients}')

In [ ]:
rpc.walletlock()

In [ ]:
# rpc = Call(getenv('RPCUSER'), getenv('RPCPASS'), 8770)

# txs = tdb.lgetall('wallet.0')
# rpc.gettransaction(txs[-1])['details']

print(rpc.gettransaction(''))

In [ ]:

# def load_db(filename): return pickledb.load(f'{filename}.json', True)
# db = load_db('addresses')


with ThreadPoolExecutor() as executor: pass
# def lexists_seq(name, data):
#     if type(data) is list | tuple:
#         for d in data: 
#             pass
#     
# def lexists_ladd(self, name, data): 
#     if not db.lexists(name, data): db.ladd(name, data)
# def save_addresses(account): rpc.getaddressesbyaccount(account)
# 
# 
# def add_list(self, name, data):
#     if not db.exists(name): db.lcreate(name)
# db.append('joshie', ['2'])
cache = []

# for address in db.lgetall('joshie'):
#     print(address)
# list_accounts(rpc)

In [ ]:
rpc = Call(getenv('RPCUSER'), getenv('RPCPASS'), 8770)

file = Path.absolute(Path('foxd_assets.json'))
db = pickledb.load(file, True)
search = 'ENO'

def is_valid_asset(asset):
    if db.exists(asset):
        print(db.get(search))
        return True
    for key, value in rpc.listassets(search, True):
        db.set(key, value)
        if key is search: return True
    return False
is_valid_asset(search)

In [ ]:
from hashlib import new
def ripemd160(data): return new('ripemd160', data)
ripemd160(b'hello').hexdigest()